<a href="https://colab.research.google.com/github/lalit-kumr/Cyber-Security-Case-Study/blob/main/Cyber%20Security%20Case%20Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##NETWORK INTRUSION DETECTION

### Your task to build network intrusion detection system to detect anamolies and attacks in the network

In [ ]:
# importing importing modules
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import imblearn

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


### Importing data

In [ ]:
smurf = pd.read_csv('https://github.com/lalit-kumr/Cyber-Security-Case-Study/raw/main/Data_of_Attack_Back_Smurf.csv')
satan = pd.read_csv('https://github.com/lalit-kumr/Cyber-Security-Case-Study/raw/main/Data_of_Attack_Back_Satan.csv')
root_kit = pd.read_csv('https://github.com/lalit-kumr/Cyber-Security-Case-Study/raw/main/Data_of_Attack_Back_RootKit.csv')
port_sweep = pd.read_csv('https://github.com/lalit-kumr/Cyber-Security-Case-Study/raw/main/Data_of_Attack_Back_PortSweep.csv')
normal = pd.read_csv('https://github.com/lalit-kumr/Cyber-Security-Case-Study/raw/main/Data_of_Attack_Back_Normal.csv')
neptune = pd.read_csv('https://github.com/lalit-kumr/Cyber-Security-Case-Study/raw/main/Data_of_Attack_Back_Neptune.csv')
nmap = pd.read_csv('https://github.com/lalit-kumr/Cyber-Security-Case-Study/raw/main/Data_of_Attack_Back_NMap.csv')
guess_pass = pd.read_csv('https://github.com/lalit-kumr/Cyber-Security-Case-Study/raw/main/Data_of_Attack_Back_GuessPassword.csv')
ftp_write = pd.read_csv('https://github.com/lalit-kumr/Cyber-Security-Case-Study/raw/main/Data_of_Attack_Back_FTPWrite.csv ')
buffer_overflow = pd.read_csv('https://github.com/lalit-kumr/Cyber-Security-Case-Study/raw/main/Data_of_Attack_Back_BufferOverflow.csv')
back = pd.read_csv('https://github.com/lalit-kumr/Cyber-Security-Case-Study/raw/main/Data_of_Attack_Back.csv')
back.head()


,attack,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_error_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,back,0.0,0,0,0.0,0.5454,0.08314,0,0,0,0.2,0,0.1,0.1,0,0,0,0,0,0,0,0,0,0.001,0.002,0.0,0.0,0.0,0.050,0.1,0.0,0.100,0.001,0.001,0.1,0,0.100,0,0.0,0.0,0.0,0.0
1,back,0.0,0,0,0.0,0.5454,0.08314,0,0,0,0.2,0,0.1,0.1,0,0,0,0,0,0,0,0,0,0.002,0.003,0.0,0.0,0.0,0.033,0.1,0.0,0.067,0.002,0.002,0.1,0,0.050,0,0.0,0.0,0.0,0.0
2,back,0.0,0,0,0.0,0.5454,0.08314,0,0,0,0.2,0,0.1,0.1,0,0,0,0,0,0,0,0,0,0.003,0.004,0.0,0.0,0.0,0.025,0.1,0.0,0.050,0.003,0.003,0.1,0,0.033,0,0.0,0.0,0.0,0.0
3,back,0.0,0,0,0.0,0.5454,0.08314,0,0,0,0.2,0,0.1,0.1,0,0,0,0,0,0,0,0,0,0.004,0.004,0.0,0.0,0.0,0.000,0.1,0.0,0.000,0.004,0.004,0.1,0,0.025,0,0.0,0.0,0.0,0.0
4,back,0.0,0,0,0.0,0.5454,0.08314,0,0,0,0.2,0,0.1,0.1,0,0,0,0,0,0,0,0,0,0.004,0.004,0.0,0.0,0.0,0.000,0.1,0.0,0.000,0.005,0.005,0.1,0,0.020,0,0.0,0.0,0.0,0.0


### Joining df's

In [ ]:
pd.concat([smurf, satan], ignore_index=True, sort=False)

,attack,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_error_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,smurf,0.0,0.02,0.09,0.0,0.01032,0.00000,0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0.0,0.001,0.001,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.146,0.001,0.001,0.002,0.001,0.0,0.000,0.0,0.000,0.0
1,smurf,0.0,0.02,0.09,0.0,0.01032,0.00000,0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0.0,0.315,0.315,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.147,0.002,0.001,0.002,0.001,0.0,0.000,0.0,0.000,0.0
2,smurf,0.0,0.02,0.09,0.0,0.01032,0.00000,0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0.0,0.316,0.316,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.148,0.003,0.002,0.002,0.002,0.0,0.000,0.0,0.000,0.0
3,smurf,0.0,0.02,0.09,0.0,0.01032,0.00000,0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0.0,0.511,0.511,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.149,0.004,0.003,0.002,0.003,0.0,0.000,0.0,0.000,0.0
4,smurf,0.0,0.02,0.09,0.0,0.01032,0.00000,0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0.0,0.511,0.511,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.150,0.005,0.003,0.002,0.003,0.0,0.000,0.0,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8021,satan,0.0,0.02,0.06,0.0,0.00020,0.00000,0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0.0,0.001,0.001,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.212,0.003,0.001,0.076,0.001,0.0,0.003,0.0,0.062,0.0
8022,satan,0.0,0.02,0.06,0.0,0.00020,0.00000,0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0.0,0.002,0.002,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.213,0.004,0.002,0.076,0.002,0.0,0.003,0.0,0.062,0.0
8023,satan,0.0,0.02,0.06,0.0,0.00020,0.00000,0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0.0,0.001,0.001,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.063,0.001,0.002,0.008,0.003,0.0,0.000,0.0,0.002,0.0
8024,satan,0.0,0.02,0.06,0.0,0.00020,0.00000,0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0.0,0.001,0.001,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.064,0.002,0.003,0.008,0.005,0.0,0.000,0.0,0.002,0.0


In [ ]:
smurf
satan
root_kit
port_sweep
normal
neptune
nmap
guess_pass
ftp_write 
buffer_oveflow
back

In [1]:
############################
# sdfsadf
# 